## 5.6 Affine/Softmaxレイヤの実装

前のセクションまでで乗算、加算、ReLU, Sigmoidといった計算レイヤの逆伝播を実装してきた。

参考： https://sge.qiita.com/yoshinari_yuto/items/e07fc89c36e123a58b49

このセクションからニューラルネットワークの逆伝播を実装していく。具体的には、行列の積（幾何学でアフィン変換とよばれる）における逆伝播を考える。

### 5.6.1 Affineレイヤ

ニューラルネットワークの順伝播は次のように実装した。

In [9]:
import numpy as np

X = np.random.rand(2) # 入力
W = np.random.rand(2, 3) # 重み
B = np.random.rand(3) # バイアス

Y = np.dot(X, W) + B

さらにこのYは活性化関数によって変換され、次の層へ伝播される。

これまでのように、ここで行った行列の積とバイアスの和を計算グラフで表してみる。

図5-24

この図は比較的単純なグラフだが、X, W, Bがスカラ値ではなく行列であることに注意して逆伝播を考えていく。

図5-25

最後のレイヤは単純に偏微分すれば良い。

$$
{\bf Y} = (y_1, y_2, y_3), \\
\frac{\partial L}{\partial {\bf Y}} =
\left ( \frac{\partial L}{\partial y_1}, \frac{\partial L}{\partial y_2}, \frac{\partial L}{\partial y_3} \right )
$$

次の加算レイヤの逆伝播はそのまま出力すればよかった。

$$
\frac{\partial L}{\partial ({\bf X} \cdot {\bf W})} = \frac{\partial L}{\partial {\bf Y}}, \\
\frac{\partial L}{\partial {\bf B}} = \frac{\partial L}{\partial {\bf Y}}
$$

最後のAffineレイヤ (アフィン変換を行う処理) は

$$
{\bf F} = {\bf X} \cdot {\bf W}
$$
と置いて、
$$
\frac{\partial L}{\partial {\bf X}} = \frac{\partial L}{\partial {\bf F}} \cdot \frac{\partial {\bf F}}{\partial {\bf X}} =
\frac{\partial L}{\partial ({\bf X} \cdot {\bf W})} \cdot \frac{\partial ({\bf X} \cdot {\bf W})}{\partial {\bf X}}
= \frac{\partial L}{\partial {\bf Y}} \cdot \frac{\partial ({\bf X} \cdot {\bf W})}{\partial {\bf X}}
$$
行列の積ができるように次元を調整して、
$$
\frac{\partial L}{\partial {\bf X}} = \frac{\partial L}{\partial {\bf Y}} \cdot {\bf W}^{\mathrm{T}}
$$

図5-26

Wについても同様に
$$
\frac{\partial L}{\partial {\bf W}} = {\bf X}^{\mathrm{T}} \cdot \frac{\partial L}{\partial {\bf Y}}
$$


### 5.6.2 バッチ版Affineレイヤ

前節で扱ったXはベクトルで、一つのデータを対象としていた。
次にN個のデータをまとめて順伝播する場合（バッチ）のAffineレイヤを考える。

図5-27

ここで、バイアス項Bの加算において行列の次元が異なるが、各データに対してWを加算することに注意する。
（N個のデータに対してAffine変換を実行すると考えれば自明）

計算例としては

In [10]:
X_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])

X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [11]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

このようにバイアスBの加算は各データに対して行われる。そのため、逆伝播の際には各データの逆伝播の値がバイアスに集約される。（そもそも次元も違うため、そうしなければ計算も合わない）

プログラムで表すと以下のようになる

In [12]:
dY = np.array([[1,2,3], [4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [13]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

以上から、Affineレイヤの実装は以下のようになる

In [15]:
class Affine:
    def __init__(self, W, b): # 慣習としてベクトルは小文字で表される
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T) # .Tは転置
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

### 5.6.3 Softmax-with-Lossレイヤ

最後に出力層のSoftmaxレイヤを実装していく。ソフトマックス関数については既に取り扱ったことがあるので、ここでの説明は省略する。

損失関数である交差エントロピー誤差 (cross entropy error) も含めて、Softmax-with-Lossレイヤという名前で実装する。
計算グラフは以下の図のようになる

図5-29

簡易的に表すと次の図のようになる

図5-30

ここでtはone-hotな教師ラベル（正解データ）である。まずCross entropy errorレイヤの逆伝播を考える。対数関数の微分は

$$
(\log (x) )' = \frac{1}{x}
$$

だったので、逆伝播を一つ一つ記入していくと下図のようになる

図A-4

よってCross entropy errorレイヤの逆伝播は

$$
\left ( -\frac{t_1}{y_1}, -\frac{t_2}{y_2}, -\frac{t_3}{y_3} \right )
$$

となる。続いてSoftmaxレイヤの逆伝播を考える。

ステップ1

まずCross entropy errorレイヤからの逆伝播の値が流れてくる。

ステップ2

最初の乗算ノードの片側（除算ノードに向かう方）は次のようになる

$$
-\frac{t_1}{y_1} \exp (a_1) = -\frac{t_1}{\frac{\exp (a_1)}{S}} \exp (a_1) = -t_1 S
$$

ステップ3

続けて除算の微分をする。この除算ノードは枝分かれして複数のノードに順伝播しており、バイアス項のときと同じように逆伝播では集約（各データを全て加算）される。
よって

$$
\left ( \frac{1}{f} \right )' = -\frac{f'}{f^2}
$$

なので、

$$
-S (t_1 + t_2 + t_3) \cdot \left ( \frac{1}{S} \right )' = S (t_1 + t_2 + t_3) \cdot \frac{1}{S^2} = \frac{1}{S} (t_1 + t_2 + t_3)
$$

ステップ4

またtはone-hotラベルであったので、その総和は1となり、このノードの逆伝播は

$$
\frac{1}{S}
$$

となる。加算ノードはそのまま流すだけ。

ステップ5

加算ノードのもう片方（expノードに向かう方）は

$$
-\frac{t_1}{y_1} \cdot \frac{1}{S} = -\frac{t_1}{\frac{\exp (a_1)}{S}} \cdot \frac{1}{S} = - \frac{t_1}{\exp(a_1)}
$$

となる。

ステップ6

最後にexpノードだが、expは微分しても変わらないので、

$$
\left ( \frac{1}{S} - \frac{t_1}{\exp(a_1)} \right ) \cdot (\exp (a_1))' = \left ( \frac{1}{S} - \frac{t_1}{\exp(a_1)} \right ) \cdot \exp (a_1)
= \frac{\exp (a_1)}{S} - t_1 = y_1 - t_1
$$

となる。以上から、順伝播の入力がa1のノードでは、逆伝播がy1-t1となることが導かれた。
逆伝播から最終的に得るベクトルは

$$
(y_1 - t_1, \, y_2 - t_2, \, y_3 - t_3)
$$

となる。これはたまたまきれいな結果が出たわけではなく、逆伝播が計算しやすくなるように交差エントロピー誤差が定義されている。

ここで具体例として、教師ラベルが (0,1,0) のようなデータに対して、Softmaxレイヤの出力が (0.3, 0.2, 0.5) 出会った場合を考える。
正解ラベルに対する確率は0.2なので、この時点では学習がまだまだ進んでいないことがわかる。
実際にこのレイヤの逆伝播は (0.3, -0.8, 0.5) となり、大きな誤差を伝播することになる。

重みを変えて学習を続け、教師ラベルが (0,1,0) のようなデータに対してSoftmaxレイヤの出力が (0.01, 0.99, 0) となった場合を考える。
この場合Softmaxレイヤからの逆伝播は (0.01, -0.01, 0) という小さな値になる。